In [ ]:
import openai

client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
         {"role": "user", "content": "회사 클라우드에서 openAI API 사용못하는 이유가 뭐지?"},
    ],
)

print(response.choices[0].message.content)

APITimeoutError: Request timed out.

In [1]:
import os

# 환경 변수가 제대로 로드되었는지 확인 (키 값은 직접 출력하지 않도록 주의)
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    print("✅ API Key가 .env 파일에서 성공적으로 로드되었습니다.")
else:
    print("❌ 오류: .env 파일에 GEMINI_API_KEY가 설정되어 있지 않습니다.")

✅ API Key가 .env 파일에서 성공적으로 로드되었습니다.


In [6]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="회사 cloud PC에서 openAI API는 사용을 못하고, gemini API 사용이 가능한 이유는 뭘까??"
)

print(response.text)   

회사 클라우드 PC에서 OpenAI API는 막혀있고 Gemini API는 사용 가능한 상황이라면, 몇 가지 복합적인 이유가 있을 수 있습니다. 주로 **회사 내부의 보안 정책, 데이터 처리 규정, IT 관리 전략, 그리고 벤더(Vendor)와의 관계**에서 기인하는 경우가 많습니다.

다음은 가능한 이유들을 종합적으로 설명해 드립니다.

1.  **보안 및 데이터 처리 정책 (Security & Data Handling Policies):**
    *   **OpenAI에 대한 초기 우려:** OpenAI가 처음 주목받기 시작했을 때, 사용자 데이터가 모델 학습에 사용될 수 있다는 우려가 있었습니다. 비록 OpenAI가 기업 사용자 데이터는 모델 학습에 사용하지 않는다고 발표했지만, 초기 정책 수립 단계에서는 이러한 불확실성 때문에 기업들이 민감한 데이터 유출을 막기 위해 차단하는 경우가 많았습니다. 회사 기밀이나 개인 정보가 외부로 나가는 것을 엄격하게 통제해야 하는 기업에게는 큰 리스크로 다가왔을 수 있습니다.
    *   **Google의 기업형 솔루션 신뢰성:** Google은 오랫동안 기업들을 대상으로 Google Cloud Platform (GCP)이라는 강력한 클라우드 인프라를 제공해 왔습니다. GCP 내에서 제공되는 Gemini API(Vertex AI 등)는 기업 고객을 위한 데이터 보안, 프라이버시, 규정 준수(Compliance)에 대한 엄격한 약관과 기능을 갖추고 있습니다. 이미 Google과 계약 관계에 있는 기업은 Gemini API를 더 신뢰할 수 있다고 판단할 수 있습니다.

2.  **IT 관리 및 통제 (IT Management & Control):**
    *   **네트워크 방화벽/프록시 설정:** 회사 네트워크는 특정 도메인이나 IP 주소로의 접근을 허용하거나 차단할 수 있습니다. OpenAI API의 엔드포인트(`api.openai.com` 등)는 차단 목록에 있을 수 있고, Gemini API의 엔드포인트(`ge

In [7]:
# 네트워크 연결 테스트
import socket
import subprocess

def test_connection(host, port=443):
    """특정 호스트의 포트 연결 테스트"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except Exception as e:
        print(f"연결 테스트 중 오류: {e}")
        return False

# OpenAI API 엔드포인트 테스트
print("=" * 50)
print("네트워크 연결 테스트")
print("=" * 50)

openai_host = "api.openai.com"
gemini_host = "generativelanguage.googleapis.com"

print(f"\n1. OpenAI API ({openai_host}):")
if test_connection(openai_host):
    print("   ✅ 연결 가능")
else:
    print("   ❌ 연결 불가 (방화벽/프록시 차단 가능)")

print(f"\n2. Gemini API ({gemini_host}):")
if test_connection(gemini_host):
    print("   ✅ 연결 가능")
else:
    print("   ❌ 연결 불가")

print("\n" + "=" * 50)
print("추가 진단: PowerShell에서 다음 명령어 실행")
print("=" * 50)
print(f"Test-NetConnection -ComputerName {openai_host} -Port 443")
print(f"Test-NetConnection -ComputerName {gemini_host} -Port 443")

네트워크 연결 테스트

1. OpenAI API (api.openai.com):
   ✅ 연결 가능

2. Gemini API (generativelanguage.googleapis.com):
   ✅ 연결 가능

추가 진단: PowerShell에서 다음 명령어 실행
Test-NetConnection -ComputerName api.openai.com -Port 443
Test-NetConnection -ComputerName generativelanguage.googleapis.com -Port 443


In [9]:
# SSL 연결 상세 진단
import ssl
import socket
from urllib.parse import urlparse

def test_ssl_handshake(host, port=443):
    """SSL 핸드셰이크 테스트"""
    print(f"\n{'='*60}")
    print(f"SSL 핸드셰이크 테스트: {host}:{port}")
    print(f"{'='*60}")
    
    try:
        # TCP 연결
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        print(f"1. TCP 연결 시도...")
        sock.connect((host, port))
        print(f"   ✅ TCP 연결 성공")
        
        # SSL 컨텍스트 생성
        context = ssl.create_default_context()
        context.check_hostname = False
        context.verify_mode = ssl.CERT_NONE  # 테스트용
        
        # SSL 핸드셰이크
        print(f"2. SSL 핸드셰이크 시도...")
        ssl_sock = context.wrap_socket(sock, server_hostname=host)
        print(f"   ✅ SSL 핸드셰이크 성공")
        print(f"   SSL 버전: {ssl_sock.version()}")
        print(f"   암호화 방식: {ssl_sock.cipher()}")
        
        ssl_sock.close()
        return True
        
    except socket.timeout:
        print(f"   ❌ 타임아웃: SSL 핸드셰이크가 완료되지 않음")
        print(f"   → 회사 방화벽이 SSL 트래픽을 차단하거나 지연시킬 수 있음")
        return False
    except ssl.SSLError as e:
        print(f"   ❌ SSL 오류: {e}")
        return False
    except Exception as e:
        print(f"   ❌ 오류: {type(e).__name__}: {e}")
        return False

# OpenAI API 엔드포인트 SSL 테스트
test_ssl_handshake("api.openai.com")



SSL 핸드셰이크 테스트: api.openai.com:443
1. TCP 연결 시도...
   ✅ TCP 연결 성공
2. SSL 핸드셰이크 시도...
   ❌ 타임아웃: SSL 핸드셰이크가 완료되지 않음
   → 회사 방화벽이 SSL 트래픽을 차단하거나 지연시킬 수 있음


False

# 🔍 문제 진단 결과 요약

## 확인된 사실
- ✅ **TCP 연결**: 성공 (443 포트 접근 가능)
- ❌ **SSL 핸드셰이크**: 타임아웃 발생
- ✅ **Gemini API**: 정상 작동

## 결론
회사 방화벽/보안 장비가 `api.openai.com`에 대한 SSL/TLS 트래픽을 차단하거나 검사 중입니다.

## 해결 방법
1. **IT 부서에 방화벽 오픈 요청** (권장)
2. 회사 인증서(CA) 사용 시도
3. Gemini API 계속 사용 (현재 가능)


In [10]:
# IT 부서 요청용 정보 생성
print("=" * 70)
print("IT 부서 방화벽 오픈 요청 정보")
print("=" * 70)
print()

print("📋 요청 내용:")
print("-" * 70)
print("OpenAI API 사용을 위한 방화벽 정책 변경 요청")
print()
print("🔍 문제 상황:")
print("  - TCP 연결(443 포트): 정상")
print("  - SSL 핸드셰이크: 타임아웃 발생")
print("  - 진단 결과: 회사 방화벽이 SSL/TLS 트래픽을 차단/검사 중")
print()
print("🌐 허용이 필요한 도메인/IP:")
print("  - api.openai.com (도메인)")
print("  - IP 주소: 172.66.0.243 (동적이므로 도메인 기반 정책 권장)")
print("  - 포트: 443 (HTTPS)")
print("  - 프로토콜: HTTPS/TLS")
print()
print("📝 기술적 세부사항:")
print("  - OpenAI API는 HTTPS를 통해서만 접근 가능")
print("  - SSL/TLS 핸드셰이크가 완료되어야 API 호출 가능")
print("  - 현재 SSL 핸드셰이크 단계에서 타임아웃 발생")
print()
print("✅ 참고사항:")
print("  - Gemini API(generativelanguage.googleapis.com)는 정상 작동 중")
print("  - Google Cloud Platform 도메인은 이미 허용되어 있음")
print("  - OpenAI API도 동일한 방식으로 허용 요청")
print()
print("=" * 70)
print()
print("📧 요청 템플릿 (이메일/티켓 시스템용):")
print("-" * 70)
print("""
제목: OpenAI API 사용을 위한 방화벽 정책 변경 요청

안녕하세요,

업무상 OpenAI API 사용이 필요하여 방화벽 정책 변경을 요청드립니다.

[문제 상황]
- TCP 연결(443 포트)은 정상적으로 이루어지지만
- SSL 핸드셰이크 단계에서 타임아웃이 발생합니다
- 진단 결과, 회사 방화벽이 SSL/TLS 트래픽을 차단하거나 검사 중인 것으로 확인되었습니다

[요청 사항]
다음 도메인에 대한 HTTPS(443 포트) 접근을 허용해 주시기 바랍니다:
- api.openai.com

[참고]
- Gemini API(generativelanguage.googleapis.com)는 현재 정상 작동 중입니다
- OpenAI API도 동일한 방식으로 허용 요청드립니다

감사합니다.
""")
print("=" * 70)


IT 부서 방화벽 오픈 요청 정보

📋 요청 내용:
----------------------------------------------------------------------
OpenAI API 사용을 위한 방화벽 정책 변경 요청

🔍 문제 상황:
  - TCP 연결(443 포트): 정상
  - SSL 핸드셰이크: 타임아웃 발생
  - 진단 결과: 회사 방화벽이 SSL/TLS 트래픽을 차단/검사 중

🌐 허용이 필요한 도메인/IP:
  - api.openai.com (도메인)
  - IP 주소: 172.66.0.243 (동적이므로 도메인 기반 정책 권장)
  - 포트: 443 (HTTPS)
  - 프로토콜: HTTPS/TLS

📝 기술적 세부사항:
  - OpenAI API는 HTTPS를 통해서만 접근 가능
  - SSL/TLS 핸드셰이크가 완료되어야 API 호출 가능
  - 현재 SSL 핸드셰이크 단계에서 타임아웃 발생

✅ 참고사항:
  - Gemini API(generativelanguage.googleapis.com)는 정상 작동 중
  - Google Cloud Platform 도메인은 이미 허용되어 있음
  - OpenAI API도 동일한 방식으로 허용 요청


📧 요청 템플릿 (이메일/티켓 시스템용):
----------------------------------------------------------------------

제목: OpenAI API 사용을 위한 방화벽 정책 변경 요청

안녕하세요,

업무상 OpenAI API 사용이 필요하여 방화벽 정책 변경을 요청드립니다.

[문제 상황]
- TCP 연결(443 포트)은 정상적으로 이루어지지만
- SSL 핸드셰이크 단계에서 타임아웃이 발생합니다
- 진단 결과, 회사 방화벽이 SSL/TLS 트래픽을 차단하거나 검사 중인 것으로 확인되었습니다

[요청 사항]
다음 도메인에 대한 HTTPS(443 포트) 접근을 허용해 주시기 바랍니다:
- api.openai.com

[참고]
- Gemini

In [ ]:
# 대안: 회사 인증서(CA)가 필요한 경우 시도
# (IT 부서에서 인증서 파일을 제공받은 경우)

import os
import httpx
import openai

# 회사 인증서 경로 (IT 부서에서 제공받은 경우)
cert_path = r"C:\Users\75574\cert.cer"  # 실제 경로로 변경 필요

print("=" * 70)
print("회사 인증서를 사용한 OpenAI API 호출 시도")
print("=" * 70)
print()

# 인증서 파일 존재 여부 확인
cert_path = r"C:\Users\75574\cert.cer"
if os.path.exists(cert_path):
    print(f"✅ 인증서 파일 발견: {cert_path}")
    print()
    
    # httpx 클라이언트 설정 (회사 인증서 사용)
    timeout = httpx.Timeout(
        connect=30.0,
        read=60.0,
        write=30.0,
        pool=10.0
    )
    
    http_client = httpx.Client(
        timeout=timeout,
        verify=cert_path,  # 회사 인증서 사용
    )
    
    client = openai.OpenAI(
        http_client=http_client,
        max_retries=2,
    )
    
    try:
        print("OpenAI API 호출 시도 중...")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": "안녕하세요!"},
            ],
            timeout=90.0,
        )
        print("\n✅ 성공! 회사 인증서로 연결되었습니다.")
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"\n❌ 오류: {type(e).__name__}: {e}")
        print("\n💡 IT 부서에 방화벽 오픈을 요청하거나 인증서 경로를 확인하세요.")
else:
    print(f"❌ 인증서 파일을 찾을 수 없습니다: {cert_path}")
    print()
    print("💡 IT 부서에 다음을 요청하세요:")
    print("   1. 회사 SSL 인증서(CA) 파일 제공")
    print("   2. 또는 방화벽 정책 변경 (api.openai.com 허용)")
    print()
    print("=" * 70)


회사 인증서를 사용한 OpenAI API 호출 시도

✅ 인증서 파일 발견: C:\Users\75574\cert.cer

OpenAI API 호출 시도 중...


In [1]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="google genAI API Key 무료 버전은 얼마나 쓸 수 있는건가?"
)

print(response.text)

Google GenAI API Key (주로 Gemini API를 말씀하시는 것으로 보입니다)의 무료 버전은 'Always Free' 티어 정책에 따라 일정량까지는 무료로 사용할 수 있습니다.

주요 무료 사용량은 다음과 같습니다 (2024년 초 기준이며, 정책은 변경될 수 있으니 항상 공식 문서를 확인하는 것이 좋습니다).

### Gemini Pro 모델 (Vertex AI 기준)

1.  **Gemini Pro (텍스트 입력):**
    *   **월별 100만 토큰까지 무료**
    *   (초당 60개 요청(RPM) 및 분당 60,000 토큰(TPM)의 제한이 있습니다.)
2.  **Gemini Pro Vision (이미지 입력):**
    *   **월별 1,000장의 이미지까지 무료**
    *   (초당 60개 요청(RPM)의 제한이 있습니다.)
3.  **Embeddings (임베딩):**
    *   **월별 100만 토큰까지 무료**

### 중요 사항:

*   **결제 계정 필요:** 무료 티어를 사용하더라도 Google Cloud에 **결제 계정을 설정해야 합니다.** 이는 오용을 방지하고, 무료 사용량을 초과했을 때 자동으로 유료 전환이 되도록 하기 위함입니다.
*   **모델 제한:** 이 무료 정책은 주로 **Gemini Pro** 모델에 적용됩니다. Gemini Ultra와 같은 고급 모델은 처음부터 유료입니다.
*   **지역별 가격:** 무료 티어를 초과하는 사용량에 대한 가격은 지역에 따라 다를 수 있습니다.
*   **Vertex AI:** 이 API는 Google Cloud의 Vertex AI 플랫폼을 통해 제공되며, GenAI Studio에서도 활용 가능합니다.
*   **정책 변경 가능성:** Google Cloud의 정책은 변경될 수 있으므로, 항상 최신 정보는 [Google Cloud Vertex AI Generative AI 가격 책정](https://cloud.google.com/vertex-ai/g

In [2]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="google genAI API Key 무료 버전은 얼마나 쓸 수 있는건가?"
)

response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""Google GenAI (Vertex AI의 일부) API는 **무료 등급(Free Tier)**을 제공하지만, 무제한은 아니며 특정 사용량까지 무료로 사용할 수 있습니다. 이 무료 사용량은 모델(Gemini, PaLM 2 등)과 사용 방식(텍스트, 이미지, 임베딩)에 따라 다릅니다.

가장 정확한 정보는 Google Cloud의 공식 가격 책정 페이지에서 확인해야 하지만, 일반적인 무료 등급의 특징은 다음과 같습니다.

### Google GenAI API 무료 등급의 일반적인 특징 (변경될 수 있음)

1.  **모델별 차등 적용:**
    *   **Gemini Pro (텍스트):** 주로 텍스트 입력 및 출력 문자 수로 측정됩니다. 예를 들어, 월별 특정량(예: 1백만 입력 문자 및 1백만 출력 문자)까지 무료로 제공될 수 있습니다. (이는 프로모션에 따라 더 많거나 적을 수 있습니다.)
    *   **Gemini Pro Vision (멀티모달):** 이미지 입력 수로 측정됩니다. 예를 들어, 월별 특정량(예: 1,000 이미지 입력)까지 무료로 제공될 수 있습니다.
    *   **Text Embedding:** 텍스트 임베딩 생성 시 문자 수로 측정됩니다. 예를 들어, 월별 1백만 문자까지 무료로 제공될 수 있습니다.
    *   **PaLM 2 (text-bison, chat-bison 등):** Gemini Pro와 유사하게 텍스트 입출력 문자 수로 측정됩니다.

2.  **측정 단위:**
    *   **텍스트:** 1,000자(문자) 단위로 측정됩니다. 입력 문자(input characters)와 출력 문자(output charac

In [5]:
from google import genai

client = genai.Client()

porompt = """
내 시스템에는 아래 함수가 있어.

'get_weather'
'get_currency'
'get_news'

이 함수들은 나라를 변수로 받음. 예를들어 get_weather('korea') 이렇게 사용함.
대답은 내가 실행할 수 있게 함수이름을 줘. 다른 아무것도 말고 함수이름과 변수만 알려줘.

이제 이렇게 질문해볼께: 대한민국 뉴스는 뭐야?
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=porompt
)

print(response.text)

get_news('korea')


In [4]:
from google import genai

client = genai.Client()
messages = []

In [6]:
def call_ai():
    # Gemini API는 OpenAI 형식의 messages를 받지 않음
    # 대화 히스토리를 하나의 문자열로 합쳐서 전달
    conversation = ""
    for msg in messages:
        role = msg.get("role", "user")
        content = msg.get("content", "")
        if role == "user":
            conversation += f"사용자: {content}\n\n"
        elif role == "assistant":
            conversation += f"AI: {content}\n\n"
        
    # 마지막 사용자 메시지만 전달 (대화 히스토리 포함)
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=conversation
    )
    
    # 응답을 messages에 추가
    messages.append({"role": "assistant", "content": response.text})
    print(f"AI: {response.text}")
    

In [3]:
while True:
    ask = input("send a msg to the LLM: ")
    if ask == 'exit':
        break
    else:
        messages.append({"role": "user", "content": ask})
        print(f"User: {ask}")
        call_ai()
        print()  # 빈 줄 추가


User: 노기현 알아?
AI: 네, 노기현 씨에 대해 알고 있습니다.

가장 잘 알려진 분은 **대한민국의 전 축구 선수**입니다.

주요 정보는 다음과 같습니다:

*   **이름:** 노기현 (Noh Ki-hyun)
*   **출생:** 1983년 1월 12일
*   **포지션:** 공격수, 미드필더
*   **주요 경력:**
    *   K리그 (전남 드래곤즈, 대구 FC, 대전 시티즌 등)
    *   유럽 리그 (벨기에 주필러 리그 등)
    *   대한민국 U-20, U-23 국가대표팀

은퇴 후에는 지도자나 해설가 등으로 활동하기도 했습니다.

혹시 다른 노기현 씨를 말씀하시는 것이라면 구체적인 정보를 알려주시면 더 정확히 찾아드릴 수 있습니다.

User: 아닌데? 진짜 인물 맞아?
AI: 아, 죄송합니다. 제가 다른 분과 혼동했던 것 같습니다.

말씀하신 노기현 님은 **IT/스타트업 분야에서 활발하게 활동하고 있는 인물**이 맞습니다.

가장 잘 알려진 분은 **IT 전문 미디어 '아웃스탠딩'의 창업자이자 대표**인 노기현 님입니다.

주요 정보는 다음과 같습니다:

*   **직업:** IT 전문 미디어 '아웃스탠딩' 대표, 칼럼니스트, 연사
*   **주요 활동:**
    *   IT 및 스타트업 업계 트렌드 분석, 칼럼 작성
    *   다양한 강연 및 세미나 활동
    *   저서 집필
*   **특징:** 복잡한 IT 기술이나 비즈니스 모델을 쉽고 재미있게 풀어내는 능력으로 많은 독자와 청중들에게 사랑받고 있습니다.
*   **저서:** '스타트업은 어떻게 세상을 바꾸는가' 등의 책을 쓰기도 했습니다.

혹시 다른 분야의 노기현 님을 찾으시는 것이라면 다시 알려주시면 정확하게 찾아드리겠습니다. 혼동을 드려 죄송합니다!

User: 거짓말하지마. 생년월일이 뭐야?
AI: 죄송합니다. 제가 또다시 혼란을 드려 신뢰를 잃게 된 점 깊이 사과드립니다. 거짓말을 하려는 의도는 전혀 없었습니다.

노기현 아웃스탠딩 대표님의 생년월일